# AirBnB Analysis - Data 2 Insights!

## Package Imports

In [36]:
import numpy as np
import pandas as pd
import chardet
from data_cleaning import set_data_types, rename_columns, print_memory # Our first module export!
import data_cleaning
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
import pymysql
from dotenv import load_dotenv
import os

In [37]:
# Listing the user defined module's functions

# dir(data_cleaning)

## User Defined Functions

*Creating modules will enable us to export our user defined functions and reuse them again and again. We have two techniques:*
* *We can import the whole module, so you can ONLY use the functions out of this moduel if you called the function using the module name first.*
* *We can use the function directly if we imported the module and the functions out of the module during the import process.*
* *Using the function name without mentioning the module name comes with cost, CONFLICT!*

## Loading & Preprocessing & Reducing Data
*In this section of the notebook we load the data, parse the date columns and reduce the memory consumption to its lowest levels through converting object columns into Pandas.Category data type, downcasting the numeric data types into the proper size depending on its minimum and maximum range of numbers, and finally standardize the columns' header names for the further processing.*

In [38]:
# Data dictionary is good for getting a sneak peak about the data then delete it
#print_memory()

listings_data_dictionary = pd.read_csv('Airbnb Data/Listings_data_dictionary.csv')

#print_memory()

# listings_data_dictionary

# del listings_data_dictionary

In [39]:
# Detecting the dataset encoding!
# This criteria checks the first 10000 bytes which is very low to detect the proper
# encoding, so it's just a sanity check!
# with open('Airbnb Data/Listings.csv', 'rb') as f:
#    result = chardet.detect(f.read(10000)) # Reading the first 10000 bytes to detect
# print(result)

In [40]:
#print_memory()

listings = pd.read_csv('Airbnb Data/Listings.csv',
                      encoding= 'latin1', # We had a problem with utf-8 encoding.
                      parse_dates= ['host_since'],
                      low_memory= False) # First step to reduce memory.
#print_memory()

# listings

In [41]:
# The analysis will be conducted on the listings ONLY in Paris.
# Reducing the dataset before implementing any other operations will reduce memory usage.
#print_memory()

listings_paris = listings.loc[listings['city'] =='Paris']

#print_memory()

In [42]:
# Exploring the memory usage of the dataset before conducting any memory reductions
listings_paris.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
Index: 64690 entries, 0 to 279711
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   listing_id                   64690 non-null  int64         
 1   name                         64627 non-null  object        
 2   host_id                      64690 non-null  int64         
 3   host_since                   64657 non-null  datetime64[ns]
 4   host_location                64522 non-null  object        
 5   host_response_time           23346 non-null  object        
 6   host_response_rate           23346 non-null  float64       
 7   host_acceptance_rate         31919 non-null  float64       
 8   host_is_superhost            64657 non-null  object        
 9   host_total_listings_count    64657 non-null  float64       
 10  host_has_profile_pic         64657 non-null  object        
 11  host_identity_verified       64657 non-null  

In [43]:
# Leveraging the user define function (set_data_types) to decrease the data set memory usage
#print_memory()

listings_sm = set_data_types(listings_paris)
listings_sm.info(memory_usage= 'deep')

#print_memory()

<class 'pandas.core.frame.DataFrame'>
Index: 64690 entries, 0 to 279711
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   listing_id                   64690 non-null  int32         
 1   name                         64627 non-null  object        
 2   host_id                      64690 non-null  int32         
 3   host_since                   64657 non-null  datetime64[ns]
 4   host_location                64522 non-null  category      
 5   host_response_time           23346 non-null  category      
 6   host_response_rate           23346 non-null  float64       
 7   host_acceptance_rate         31919 non-null  float64       
 8   host_is_superhost            64657 non-null  category      
 9   host_total_listings_count    64657 non-null  float64       
 10  host_has_profile_pic         64657 non-null  category      
 11  host_identity_verified       64657 non-null  

*Reduced more than 60% of the memory reserved for the dataset through setting the proper data type for each column*

In [44]:
listings_clean = rename_columns(listings_sm)

# listings_clean.columns

In [45]:
reviews_data_dictionary = pd.read_csv('Airbnb Data/Reviews_data_dictionary.csv',
                                     encoding= 'latin1')
# reviews_data_dictionary.head()

# del reviews_data_dictionary

In [46]:
reviews = pd.read_csv('Airbnb Data/Reviews.csv',
                     parse_dates= ['date'])

# reviews.head()

In [47]:
reviews.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5373143 entries, 0 to 5373142
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   listing_id   int64         
 1   review_id    int64         
 2   date         datetime64[ns]
 3   reviewer_id  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 164.0 MB


In [48]:
reviews_sm = set_data_types(reviews)
reviews_sm.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5373143 entries, 0 to 5373142
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   listing_id   int32         
 1   review_id    int32         
 2   date         datetime64[ns]
 3   reviewer_id  int32         
dtypes: datetime64[ns](1), int32(3)
memory usage: 102.5 MB


In [49]:
reviews_clean = rename_columns(reviews_sm)

# reviews_clean.columns

In [50]:
print_memory()

del listings_data_dictionary
del reviews_data_dictionary
del listings
del listings_paris
del listings_sm
del reviews
del reviews_sm

print_memory()

Current memory usage: 1807.81 MB
Current memory usage: 1584.98 MB


In [51]:
listings_clean.shape

(64690, 33)

In [52]:
listings_clean.isna().sum().loc[listings_clean.isna().sum() > 0]

Name                            63
HostSince                       33
HostLocation                   168
HostResponseTime             41344
HostResponseRate             41344
HostAcceptanceRate           32771
HostIsSuperhost                 33
HostTotalListingsCount          33
HostHasProfilePic               33
HostIdentityVerified            33
District                     64690
Bedrooms                     13404
ReviewScoresRating           16654
ReviewScoresAccuracy         16701
ReviewScoresCleanliness      16692
ReviewScoresCheckin          16718
ReviewScoresCommunication    16699
ReviewScoresLocation         16719
ReviewScoresValue            16718
dtype: int64

**How are we going to deal with these null values?**
*We are going to categorize these null values into four categories:*
* *The first category is the low impact, minor data quality issues that appears in name of the listing (63) null value, host since date (33) and host location (168) null values. All these values can be dropped without impact as we should not impute or keep. The total rows dropped due to this step is 231 rows which represents 0.35%*

In [53]:
listings_clean = listings_clean.dropna(
    subset= ['Name', 'HostSince', 'HostLocation', 'HostIsSuperhost',
            'HostTotalListingsCount', 'HostHasProfilePic', 'HostIdentityVerified'])

In [54]:
listings_clean.isna().sum().loc[listings_clean.isna().sum() > 0]

HostResponseTime             41140
HostResponseRate             41140
HostAcceptanceRate           32581
District                     64459
Bedrooms                     13337
ReviewScoresRating           16546
ReviewScoresAccuracy         16593
ReviewScoresCleanliness      16584
ReviewScoresCheckin          16610
ReviewScoresCommunication    16591
ReviewScoresLocation         16611
ReviewScoresValue            16610
dtype: int64

* *The second group of null values is the district column which has almost 100 % of its value as null. So, dropping the column is the best option and we can depend on neighburhood!*

In [55]:
listings_clean.drop('District', axis= 1, inplace= True)

In [56]:
listings_clean.isna().sum().loc[listings_clean.isna().sum() > 0]

HostResponseTime             41140
HostResponseRate             41140
HostAcceptanceRate           32581
Bedrooms                     13337
ReviewScoresRating           16546
ReviewScoresAccuracy         16593
ReviewScoresCleanliness      16584
ReviewScoresCheckin          16610
ReviewScoresCommunication    16591
ReviewScoresLocation         16611
ReviewScoresValue            16610
dtype: int64

*The third category of null values are the meaningful nulls! Regarding the Bedrooms it means that this is a studio apartment where there is no bedrooms! So, this should be imputed to zero to separte such peroperites from bedrooms = 1.*

In [57]:
listings_clean.fillna({'Bedrooms': 0}, inplace= True)

In [58]:
listings_clean.isna().sum().loc[listings_clean.isna().sum() > 0]

HostResponseTime             41140
HostResponseRate             41140
HostAcceptanceRate           32581
ReviewScoresRating           16546
ReviewScoresAccuracy         16593
ReviewScoresCleanliness      16584
ReviewScoresCheckin          16610
ReviewScoresCommunication    16591
ReviewScoresLocation         16611
ReviewScoresValue            16610
dtype: int64

*The fourth group is the HostResponseTime which shall be imputed to 'N/A' or 'New Host' as it seems that this host did not receive any bookings yet, so his response time has not been monitored yet.*

In [59]:
# We need to the 'New Host' category first to allow python to let the null values filled with it
listings_clean['HostResponseTime'] = listings_clean['HostResponseTime'].cat.add_categories(['New Host'])

In [60]:
listings_clean.fillna({'HostResponseTime': 'New Host'}, inplace= True)

In [61]:
listings_clean.isna().sum().loc[listings_clean.isna().sum() > 0]

HostResponseRate             41140
HostAcceptanceRate           32581
ReviewScoresRating           16546
ReviewScoresAccuracy         16593
ReviewScoresCleanliness      16584
ReviewScoresCheckin          16610
ReviewScoresCommunication    16591
ReviewScoresLocation         16611
ReviewScoresValue            16610
dtype: int64

In [62]:
num_cols = listings_clean.select_dtypes(include= ['number']).columns
num_cols

Index(['ListingId', 'HostId', 'HostResponseRate', 'HostAcceptanceRate',
       'HostTotalListingsCount', 'Latitude', 'Longitude', 'Accommodates',
       'Bedrooms', 'Price', 'MinimumNights', 'MaximumNights',
       'ReviewScoresRating', 'ReviewScoresAccuracy', 'ReviewScoresCleanliness',
       'ReviewScoresCheckin', 'ReviewScoresCommunication',
       'ReviewScoresLocation', 'ReviewScoresValue'],
      dtype='object')

In [63]:
listings_clean[num_cols] = listings_clean[num_cols].fillna(0)
listings_clean.isna().sum().loc[listings_clean.isna().sum() > 0]

Series([], dtype: int64)

In [64]:
listings_clean.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
Index: 64459 entries, 0 to 279711
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   ListingId                  64459 non-null  int32         
 1   Name                       64459 non-null  object        
 2   HostId                     64459 non-null  int32         
 3   HostSince                  64459 non-null  datetime64[ns]
 4   HostLocation               64459 non-null  category      
 5   HostResponseTime           64459 non-null  category      
 6   HostResponseRate           64459 non-null  float64       
 7   HostAcceptanceRate         64459 non-null  float64       
 8   HostIsSuperhost            64459 non-null  category      
 9   HostTotalListingsCount     64459 non-null  float64       
 10  HostHasProfilePic          64459 non-null  category      
 11  HostIdentityVerified       64459 non-null  category      
 12  Neighbou

In [65]:
reviews_clean.isna().sum()

ListingId     0
ReviewId      0
Date          0
ReviewerId    0
dtype: int64

## Data Exploration
*In this section we are going to explore the dataset and start building our analysis framework!*

### Data Warehouse Creation
*After cleaning the data we need to build an ETL pipeline for importing, cleaning, manipulating, and reshaping data then pushing all the data into a star schema data warehouse on MySQL server.*
*The dataset has listings dimension, host dimension, should have date dimension, and finally the fact table is the reivew table*
* *The listing table should have the properies of the listings such as listing id, property type, etc.*
* *The host table should have the name, response time, etc.*
* *The date dimension should have data range that covers the whole period of the review table from start to end*
* *Finally, the fact table will have all the review details plus the listing id, host id, and datekey*

In [66]:
# These are the columns that will be mapped to the host table
host_cols = listings_clean.columns[listings_clean.columns.str.startswith('Host')]

In [67]:
# These are the columns that shall be mapped to the listing table
listing_cols = listings_clean.columns.difference(host_cols)

In [68]:
reviews_clean.head()

,ListingId,ReviewId,Date,ReviewerId
0,11798,330265172,2018-09-30,11863072
1,15383,330103585,2018-09-30,39147453
2,16455,329985788,2018-09-30,1125378
3,17919,330016899,2018-09-30,172717984
4,26827,329995638,2018-09-30,17542859


In [74]:
# Connecting to the MySQL server using the credntials from .conifg file

load_dotenv()

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}")


ValueError: invalid literal for int() with base 10: 'None'